In [ ]:
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

data_dir = "../data/"

def save_pagecontent(driver, filename): 
    content = driver.page_source
    with open(data_dir + filename, "w") as file:
        file.write(content)

# SZ

In [ ]:

driver = webdriver.Chrome()
driver.get("https://archiv.szarchiv.de/Portal/restricted/ExtendedResultList.act")
time.sleep(3)
#driver.find_element_by_xpath('//*[@id="searchTerm"]')

inputElement = driver.find_element_by_xpath('//*[@id="searchTerm"]')
inputElement.send_keys('jemen')
inputElement.send_keys(Keys.ENTER)

time.sleep(5)

dateElement = driver.find_element_by_xpath('//*[@id="dateChip"]/h4')
dateElement.click()
time.sleep(2)
fromElement = driver.find_element_by_xpath('//*[@id="fromDate"]')
fromElement.send_keys(Keys.CONTROL,'a')
time.sleep(1)
fromElement.send_keys('25.03.2015')
fromElement.send_keys(Keys.ENTER)

time.sleep(1)

for page in range(1,59):
    save_pagecontent(driver, 'sz/search-raw/'+str(page)+'.html')
    next = driver.find_element_by_css_selector('#articleSearchDIV > div.resultListNavRow.flex_x_no_wrap.flex_x_align_center > div > span.icon.iconGo.iconGoNext') 
    next.click()
    time.sleep(5)


save_pagecontent(driver, 'sz/search-raw/'+str(58)+'.html')

# ZEIT ONLINE

In [ ]:
driver = webdriver.Chrome()
driver.get("https://www.wiso-net.de/dosearch?_searchOnlyInAbstractField=&_searchOnlyInTitleField=&explicitSearch=true&q=jemen&dbShortcut=ZEIO&searchMask=6298&TI%2CUT%2CDZ%2CBT%2COT%2CSL=&AU=&MM%2COW%2CUF%2CMF%2CAO%2CTP%2CVM%2CNN%2CNJ%2CKV%2CZ2=&CO%2CC2%2CTA%2CKA%2CVA%2CZ1=&CT%2CDE%2CZ4%2CKW=&BR%2CGW%2CN1%2CN2%2CNC%2CND%2CSC%2CWZ%2CZ5%2CAI%2CBC%2CKN%2CTN%2CVN%2CK0%2CB4%2CNW%2CVH=&Z3%2CCN%2CCE%2CKC%2CTC%2CVC=&timeFilterType=on&DT_from=25.03.2015&DT_to=29.7.2022&x=83&y=8")
time.sleep(3)
#driver.find_element_by_xpath('//*[@id="searchTerm"]')

for page in range(1,25):
    save_pagecontent(driver, 'zeit/search-raw/'+str(page)+'.html')
    next = driver.find_element_by_css_selector('#content > div.divRowSearchResult.moduleBackgroundColor.clearfix.noprint > div.innerModulePager > a.nextLink') 
    next.click()
    time.sleep(5)


save_pagecontent(driver, 'zeit/search-raw/'+str(25)+'.html')



# SPIEGEL ONLINE

In [ ]:
driver = webdriver.Chrome()
driver.get("https://www.wiso-net.de/dosearch?_searchOnlyInAbstractField=&_searchOnlyInTitleField=&explicitSearch=true&q=jemen&dbShortcut=SPON&searchMask=6233&TI%2CUT%2CDZ%2CBT%2COT%2CSL=&AU=&MM%2COW%2CUF%2CMF%2CAO%2CTP%2CVM%2CNN%2CNJ%2CKV%2CZ2=&CO%2CC2%2CTA%2CKA%2CVA%2CZ1=&CT%2CDE%2CZ4%2CKW=&BR%2CGW%2CN1%2CN2%2CNC%2CND%2CSC%2CWZ%2CZ5%2CAI%2CBC%2CKN%2CTN%2CVN%2CK0%2CB4%2CNW%2CVH=&Z3%2CCN%2CCE%2CKC%2CTC%2CVC=&timeFilterType=on&DT_from=25.03.2015&DT_to=29.7.2022&x=93&y=9")
time.sleep(3)
#driver.find_element_by_xpath('//*[@id="searchTerm"]')

for page in range(1,54):
    save_pagecontent(driver, 'spiegel/search-raw/'+str(page)+'.html')
    next = driver.find_element_by_css_selector('#content > div.divRowSearchResult.moduleBackgroundColor.clearfix.noprint > div.innerModulePager > a.nextLink') 
    next.click()
    time.sleep(4)


save_pagecontent(driver, 'spiegel/search-raw/'+str(54)+'.html')
